In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from cobra.io import read_sbml_model
model = read_sbml_model('iML1515.xml.gz')
model

Name,iML1515
Memory address,0x07fb018c6aca0
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [2]:
from cobra import Reaction, Metabolite
sucd = Reaction('SUCD') #CoA-dependent succinate semialdehyde dehydrogenase
succinyl_semialdehyde = Metabolite(id='succinyl_semialdehyde_c', compartment='c')

sucd.add_metabolites({model.metabolites.h_c: 1,
                              model.metabolites.nadh_c: -1,
                              model.metabolites.succoa_c: -1,
                              succinyl_semialdehyde: 1, #new one
                              model.metabolites.coa_c: 1,
                              model.metabolites.nad_c: 1
                             })
print(sucd.build_reaction_string())
model.add_reactions([sucd])
model.reactions.SUCD

nadh_c + succoa_c --> coa_c + h_c + nad_c + succinyl_semialdehyde_c


Reaction identifier,SUCD
Name,
Memory address,0x07fb000fba550
Stoichiometry,nadh_c + succoa_c --> coa_c + h_c + nad_c + succinyl_semialdehyde_c Nicotinamide adenine dinucleotide - reduced + Succinyl-CoA --> Coenzyme A + H+ + Nicotinamide adenine dinucleotide +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [3]:
from cobra import Reaction, Metabolite
fhbd = Reaction('FHBD') #4-hydroxybutyrate dehydrogenase
four_hydroxybutyrate = Metabolite(id='four_hydroxybutyrate_c', compartment='c')

fhbd.add_metabolites({model.metabolites.h_c: 1,
                              model.metabolites.nadh_c: -1,
                              succinyl_semialdehyde: -1, 
                              four_hydroxybutyrate: 1, #new one
                              model.metabolites.nad_c: 1
                             })
print(fhbd.build_reaction_string())
model.add_reactions([fhbd])
model.reactions.FHBD

nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c


Reaction identifier,FHBD
Name,
Memory address,0x07fb000fba430
Stoichiometry,nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c Nicotinamide adenine dinucleotide - reduced + --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [4]:
from cobra import Reaction, Metabolite
hbd_coa_t = Reaction('HBCT') #4-hydroxybutyryl-CoA transferase
four_hbcoa = Metabolite(id='four_hbcoa_c', compartment='c')
hbd_coa_t.add_metabolites({model.metabolites.accoa_c: -1,
                              model.metabolites.ac_c: 1,
                              four_hbcoa: 1, #new one
                              four_hydroxybutyrate: -1
                             })
print(hbd_coa_t.build_reaction_string())
model.add_reactions([hbd_coa_t])
model.reactions.HBCT

accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c


Reaction identifier,HBCT
Name,
Memory address,0x07fb000fbaaf0
Stoichiometry,accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c Acetyl-CoA + --> Acetate +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [5]:
from cobra import Reaction, Metabolite
hbd_coa_r = Reaction('HBCR') #4-hydroxybutyryl-CoA reductase
four_hbaldehyde = Metabolite(id='four_hbaldehyde_c', compartment='c')
hbd_coa_r.add_metabolites({model.metabolites.h_c: 1,
                              model.metabolites.nadh_c: -1,
                              four_hbaldehyde: 1, #new one
                              four_hbcoa: -1,
                              model.metabolites.nad_c: 1,
                              model.metabolites.coa_c: 1
                             })
print(hbd_coa_r.build_reaction_string())
model.add_reactions([hbd_coa_r])
model.reactions.HBCR

four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c


Reaction identifier,HBCR
Name,
Memory address,0x07fb000fb8a00
Stoichiometry,four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> Coenzyme A + + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [6]:
from cobra import Reaction, Metabolite
adh= Reaction('ADH') #alcohol dehydrogenase
bdo = Metabolite(id='bdo_c', compartment='c')
adh.add_metabolites({model.metabolites.h_c: 1,
                              model.metabolites.nadh_c: -1,
                              four_hbaldehyde: -1,
                              bdo: 1, #new one
                              model.metabolites.nad_c: 1,
                             })
print(adh.build_reaction_string())
model.add_reactions([adh])
model.reactions.ADH

four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c


Reaction identifier,ADH
Name,
Memory address,0x07fb000fb84f0
Stoichiometry,four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [7]:
bdo_exchange = Reaction('EX_bdo')
bdo_exchange.add_metabolites({model.metabolites.bdo_c: -1})
model.add_reaction(bdo_exchange)
model.add_boundary(model.metabolites.bdo_c, type='demand')

Reaction identifier,DM_bdo_c
Name,demand
Memory address,0x07fb000fba9d0
Stoichiometry,bdo_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [40]:
with model:
    model.objective=model.reactions.ADH
    optimized = float(model.optimize().objective_value)
    print(optimized)

9.605894736842112


In [55]:
uptake = model.reactions.EX_glc__D_e.flux #uptake <--
print(uptake)

-10.0


In [47]:
#input text
Models_yield = dict()
Models_yield['one']= optimized
 
#open file
file = open("Yield calc.txt", "w")
 
#convert variable to string
str = repr(Models_yield)
file.write("Models_yield = " + str + "\n")
 
#close file
file.close()
 
f = open('Yield calc.txt', 'r')
if f.mode=='r':
    contents= f.read()
    



In [58]:
(-1)*((Models_yield['one'])/uptake) # it is multiplied by -1 because the reaction is an uptake which means thats goes for the right to the left

0.9605894736842112